### กลุ่ม Diabetes Prediction
1. 6610402205 นายรักษิต รุ่งรัตนไชย หมู่ 1
2. 6610402132 นายบวรรัตน์ ตั้งนรารัชชกิจ หมู่ 1
3. 6610401985 นายไชยวัตน์ หนูวัฒนา หมู่ 1

### วัตถุประสงค์ของระบบต้นแบบ

กลุ่มของพวกเราเล็งเห็นถึงความสำคัญทางด้านสุขภาพของประชาชนคนไทย ในปัจจุบันคนไทยประสบกับปัญหาสุขภาพมากมายแต่หนึ่งปัญหาที่สร้างความเสียหายให้กับสุขภาพ และคนไทยเป็นกันมากที่สุดนั่นคือโรคเบาหวาน พฤติกรรมการใช้ชีวิตของคนไทยในสมัยนี้ก็เป็นปัจจัยหนึ่งที่ทำให้ปริมาณผู้ป่วยโรคเบาหวานเพิ่มมากยิ่งขึ้น พวกเราจึงอยากทำระบบต้นแบบที่ช่วยชี้แนะหรือแนะนำผู้ที่มาใช้งานระบบต้นแบบของเราว่ามีความเสี่ยงที่จะเป็นโรคเบาหวานหรือไม่เพื่อให้เกิดการตระหนักและระมัดระวังในพฤติกรรมการใช้ชีวิตของตนเองซึ่งระบบต้นแบบของเราไม่ได้แม่นยำ 100 เปอร์เซ็นต์ ไม่สามารถใช้วินิจฉัยโรคได้แต่ใช้ประเมินความเสี่ยงเบื่องต้นของผู้ที่มาใช้งานระบบได้หากมีความเสี่ยงก็สามารถให้แพทย์วินิจัยอย่างถูกต้องและแม่นยำต่อไป

### ลิงค์ไปยังข้อมูลที่จะใช้ในระบบต้นแบบ

Link to data: https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset

In [1]:
import pandas as pd

diabetes = pd.read_csv("diabetes_binary_5050split_health_indicators_BRFSS2015.csv")

ตรวจสอบข้อมูลสูญหาย (missing value) พบว่าไม่มีข้อมูลสูญหาย

In [2]:
diabetes.isna().sum()

Diabetes_binary         0
HighBP                  0
HighChol                0
CholCheck               0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
dtype: int64

จัดการกับข้อมูลที่ซ้ำซ้อน

In [3]:
diabetes.duplicated().sum()

np.int64(1635)

In [4]:
diabetes.drop_duplicates(inplace=True)

แบ่งชุดข้อมูลเรียนรู้(train), ชุดข้อมูลทดสอบ(test), และชุดข้อมูลตรวจสอบ(validation)

In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(diabetes, test_size = 0.3, stratify = diabetes.Diabetes_binary, random_state = 1234)
test, validation = train_test_split(test, test_size = 0.3, stratify = test.Diabetes_binary, random_state = 1234)

features = ["GenHlth", "BMI", "HeartDiseaseorAttack", "DiffWalk", "HvyAlcoholConsump"]
label = "Diabetes_binary"

features ที่นำมาใช้นั้นผ่านกระบวนการ feature importance จากครั้งก่อนๆแล้วซึ่งสรุปว่าเป็น feature ที่ส่งผลต่อประสิทธิภาพของโมเดลมากที่สุด

In [6]:
len(train), len(test), len(validation)

(48339, 14502, 6216)

ตรวจสอบความสมดุลของข้อมูล

In [7]:
train.Diabetes_binary.value_counts()

Diabetes_binary
1.0    24567
0.0    23772
Name: count, dtype: int64

จะเห็นได้ว่าชุดข้อมูลเรียนรู้ขาดความสมดุล ดังนั้นต้องทำให้ข้อมูลสมดุลกันก่อน ด้วยการทำ Undersampling (ลดปริมาณข้อมูลของกลุ่มที่มีจำนวนเยอะ)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
resampler = RandomUnderSampler(random_state=1234)

train[features], train[label] = resampler.fit_resample(train[features], train[label])
train.dropna(inplace=True)

train[features].head()

,GenHlth,BMI,HeartDiseaseorAttack,DiffWalk,HvyAlcoholConsump
74322,2.0,32.0,0.0,0.0,0.0
132795,2.0,27.0,0.0,0.0,0.0
84727,4.0,30.0,0.0,0.0,0.0
204565,4.0,27.0,0.0,0.0,0.0
241574,1.0,29.0,0.0,0.0,0.0


In [ ]:
train.Diabetes_binary.value_counts()

Diabetes_binary
1.0    24568
0.0    24568
Name: count, dtype: int64

#### พัฒนาโมเดล (model devlopment) และทำ feature engineering ใน pipeline

สร้าง pipeline ของโมเดล Logistic Regression

In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LogisticRegression

def create_pipline_logistic():
    return make_pipeline(
        make_column_transformer(
            (KBinsDiscretizer(encode="ordinal", strategy="quantile"), ["BMI"]),
            remainder="passthrough"
        ),
        LogisticRegression(max_iter=500)
    )

logistic_pipeline = create_pipline_logistic()
logistic_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('kbinsdiscretizer',
                                                  KBinsDiscretizer(encode='ordinal'),
                                                  ['BMI'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

In [9]:
%time logistic_pipeline.fit(train[features], train[label])

logistic_train_score = logistic_pipeline.score(train[features], train[label])
logistic_validation_score = logistic_pipeline.score(validation[features], validation[label])
print(f"Logistic Train Score: %.3f" % logistic_train_score) 
print(f"Logistic Validation Score: %.3f" % logistic_validation_score) 

CPU times: user 51.5 ms, sys: 7.19 ms, total: 58.7 ms
Wall time: 72.5 ms
Logistic Train Score: 0.702
Logistic Validation Score: 0.712


#### ทำ hyperparameter tuning

In [10]:
logistic_pipeline.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('kbinsdiscretizer',
                                    KBinsDiscretizer(encode='ordinal'), ['BMI'])])),
  ('logisticregression', LogisticRegression(max_iter=500))],
 'verbose': False,
 'columntransformer': ColumnTransformer(remainder='passthrough',
                   transformers=[('kbinsdiscretizer',
                                  KBinsDiscretizer(encode='ordinal'), ['BMI'])]),
 'logisticregression': LogisticRegression(max_iter=500),
 'columntransformer__force_int_remainder_cols': True,
 'columntransformer__n_jobs': None,
 'columntransformer__remainder': 'passthrough',
 'columntransformer__sparse_threshold': 0.3,
 'columntransformer__transformer_weights': None,
 'columntransformer__transformers': [('kbinsdiscretizer',
   KBinsDiscretizer(encode='ordinal'),
   ['BMI'])],
 'columntransformer__verbose': False,
 'columntransformer__verbose_feature_names_o

In [11]:
from sklearn.model_selection import RandomizedSearchCV

import warnings
warnings.filterwarnings("ignore")
pipeline = create_pipline_logistic() 

param_dist = {
     'columntransformer__kbinsdiscretizer__strategy': ['uniform', 'quantile', 'kmeans'],
     'columntransformer__kbinsdiscretizer__encode': ['ordinal', 'onehot', 'onehot-dense'],
     'columntransformer__kbinsdiscretizer__n_bins': [5, 10, 20],
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'logisticregression__penalty': ['none', 'l1', 'l2', 'elasticnet'],
    'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'logisticregression__max_iter': [100, 200, 300],
    'logisticregression__tol': [1e-4, 1e-3, 1e-2]
}

search = RandomizedSearchCV(pipeline, param_dist, n_iter=20, random_state=0)
search.fit(train[features], train[label])
search.best_params_, f"{search.best_score_:.3f}"

({'logisticregression__tol': 0.01,
  'logisticregression__solver': 'lbfgs',
  'logisticregression__penalty': 'l2',
  'logisticregression__max_iter': 300,
  'logisticregression__C': 0.1,
  'columntransformer__kbinsdiscretizer__strategy': 'quantile',
  'columntransformer__kbinsdiscretizer__n_bins': 10,
  'columntransformer__kbinsdiscretizer__encode': 'onehot-dense'},
 '0.705')

### ใช้ baseline และ วิธีการประเมินอะไรได้บ้าง อย่างไร มีค่าเป็นเท่าไหร่

สร้างโมเดล baseline จากฟีเจอร์ที่ได้มาจากการทำ feature importance

1. สร้าง baseline แบบ random โดยทุกคำตอบมีความน่าจะเป็นที่จะถูกสุ่มมาเท่าๆกัน (uniform)

In [12]:
# Dummy ยังไม่ได้ทำ feature engineer
from sklearn.dummy import DummyClassifier

baseline = DummyClassifier(strategy="uniform", random_state=1234)
# baseline = DummyClassifier(strategy="stratified", random_state=1234)
# baseline = DummyClassifier(strategy="most_frequent", random_state=1234)
# baseline = DummyClassifier(strategy="constant", constant=0, random_state=1234)

baseline.fit(train[features], train.Diabetes_binary)

train_score = baseline.score(train[features], train.Diabetes_binary)
validation_score = baseline.score(validation[features], validation.Diabetes_binary) 

print(f"train score: {train_score:.3f}")
print(f"validation score: {validation_score:.3f}")

train score: 0.501
validation score: 0.503


2. สร้าง baseline แบบ random โดยทุกคำตอบมีความน่าจะเป็นที่จะถูกสุ่มมาตามสัดส่วนของคำตอบที่มี (stratified)

In [13]:
# Dummy ยังไม่ได้ทำ feature engineer
from sklearn.dummy import DummyClassifier

# baseline = DummyClassifier(strategy="uniform", random_state=1234)
baseline = DummyClassifier(strategy="stratified", random_state=1234)
# baseline = DummyClassifier(strategy="most_frequent", random_state=1234)
# baseline = DummyClassifier(strategy="constant", constant=0, random_state=1234)

baseline.fit(train[features], train.Diabetes_binary)

train_score = baseline.score(train[features], train.Diabetes_binary)
validation_score = baseline.score(validation[features], validation.Diabetes_binary) 

print(f"train score: {train_score:.3f}")
print(f"validation score: {validation_score:.3f}")

train score: 0.501
validation score: 0.490


3. สร้าง baseline แบบ zero rule baseline โดยคำตอบจะมีค่าเดียวคือคำตอบที่มีค่ามากที่สุดในชุดการเรียนรู้ (most_frequent) 

In [14]:
# Dummy ยังไม่ได้ทำ feature engineer
from sklearn.dummy import DummyClassifier

# baseline = DummyClassifier(strategy="uniform", random_state=1234)
# baseline = DummyClassifier(strategy="stratified", random_state=1234)
baseline = DummyClassifier(strategy="most_frequent", random_state=1234)
# baseline = DummyClassifier(strategy="constant", constant=0, random_state=1234)

baseline.fit(train[features], train.Diabetes_binary)

train_score = baseline.score(train[features], train.Diabetes_binary)
validation_score = baseline.score(validation[features], validation.Diabetes_binary) 

print(f"train score: {train_score:.3f}")
print(f"validation score: {validation_score:.3f}")

train score: 0.508
validation score: 0.508


4. สร้าง baseline แบบ zero rule baseline โดยทุกคำตอบมีคำตอบเดียวคือค่าคงที่ที่เรากำหนดให้ (constant)

In [15]:
# Dummy ยังไม่ได้ทำ feature engineer
from sklearn.dummy import DummyClassifier

# baseline = DummyClassifier(strategy="uniform", random_state=1234)
# baseline = DummyClassifier(strategy="stratified", random_state=1234)
# baseline = DummyClassifier(strategy="most_frequent", random_state=1234)
baseline = DummyClassifier(strategy="constant", constant=0, random_state=1234)

baseline.fit(train[features], train.Diabetes_binary)

train_score = baseline.score(train[features], train.Diabetes_binary)
validation_score = baseline.score(validation[features], validation.Diabetes_binary) 

print(f"train score: {train_score:.3f}")
print(f"validation score: {validation_score:.3f}")

train score: 0.492
validation score: 0.492


#### ประเมินผลโมเดล (model evaluation)

classification report

In [16]:
from sklearn.metrics import classification_report

y_pred = search.best_estimator_.predict(validation[features])
print(classification_report(validation[label], y_pred))

              precision    recall  f1-score   support

         0.0       0.71      0.71      0.71      3057
         1.0       0.72      0.71      0.72      3159

    accuracy                           0.71      6216
   macro avg       0.71      0.71      0.71      6216
weighted avg       0.71      0.71      0.71      6216



ทำ cross validation

In [17]:
from sklearn.model_selection import cross_val_score

cross_val_logistic_pipeline = create_pipline_logistic()
cross_val_logistic_pipeline.set_params(**search.best_params_)

%time cv_scores = cross_val_score(cross_val_logistic_pipeline, train[features], train[label], cv = 5)
print(f"cross validation score: %.3f" % cv_scores.mean())

CPU times: user 2.02 s, sys: 11.2 ms, total: 2.03 s
Wall time: 224 ms
cross validation score: 0.705


ทำ Model Calibration

In [18]:
from sklearn.calibration import CalibratedClassifierCV

calibration_logistic_pipeline = create_pipline_logistic()
calibration_logistic_pipeline.set_params(**search.best_params_)

calibration_logistic_pipeline = CalibratedClassifierCV(estimator=calibration_logistic_pipeline, cv = 5)
calibration_logistic_pipeline.fit(train[features], train[label])

train_score = calibration_logistic_pipeline.score(train[features], train[label])
valid_score = calibration_logistic_pipeline.score(validation[features], validation[label])

print(f"Train score: %.3f" % train_score)
print(f"Validation score: %.3f" % valid_score)

Train score: 0.705
Validation score: 0.711


### training score, test score ปัจจุบันมีค่าเท่าไหร่

In [19]:
train_score = calibration_logistic_pipeline.score(train[features], train[label])
test_score = calibration_logistic_pipeline.score(test[features], test[label])

print(f"Train score: %.3f" % train_score )
print(f"Test score: %.3f" % test_score)

Train score: 0.705
Test score: 0.699


สุ่มตัวอย่างจากข้อมูลทดสอบมาทดสอบโมเดล

In [20]:
sample = test.sample(random_state=1234)
sample[features]

,GenHlth,BMI,HeartDiseaseorAttack,DiffWalk,HvyAlcoholConsump
54623,2.0,26.0,0.0,0.0,0.0


Invariance Tests ทดสอบเปลี่ยนแปลงข้อมูลเล็กๆน้อย ซึ่งไม่ทำให้ผลทำนายเปลี่ยนแปลง จากการทดสอบเปลี่ยนค่าในฟีเจอร์ GenHlth เล็กน้อยจะเห็นได้ว่าผลลัพธ์การทำนายไม่เปลี่ยนแปลง (ทำนายว่าไม่เป็นเบาหวานเหมือนเดิม)

In [21]:
sample["GenHlth"] = 3.0

logistic_pipeline.predict(sample)

array([0.])

Directional Expectations Tests ทดสอบเปลี่ยนแปลงข้อมูลมากๆ ทำให้ผลการทำนายเปลี่ยนแปลงอย่างชัดเจน จากการทดสอบเปลี่ยนค่าในฟีเจอร์ GenHlth มากๆจะเห็นได้ว่าผลลัพธ์การทำนายเปลี่ยนแปลง (จากทำนายว่าไม่เป็นเบาหวานเปลี่ยนเป็นเบาหวาน)

In [22]:
sample["GenHlth"] = 5.0

logistic_pipeline.predict(sample)

array([1.])

### จากการประเมินด้วยวิธีต่าง ๆ สามารถนำโมเดลไปใช้งานจริงได้แล้วหรือไม่ เพราะเหตุใด

จากการประเมินโมเดลด้วยวิธีต่างๆทั้งหมดแล้วพบว่าโมเดลตอนน้ียังไม่สามารถใช้จริงได้ เนื่องจาก มีความแม่นยำต่ำกว่า baseline แบบ Most-Frequent ซึ่งอยู่ที่ 0.87 แต่โมเดลมีความแม่นยำที่ 0.7 จึงยังไม่สามารถนำไปใช้จริงได้ เหตุผลมาจาก ข้อมูลมีความ Imbalance มากๆแต่แรก และเราแบ่งชุดข้อมูลโดยวิธีแบบความน่าจะเป็น Stratified ทำให้ข้อมูลในชุดแต่ละชุดมีความ Imbalance แต่พวกเราก็ได้แก้ไขโดยการ undersampler ข้อมูลชุดฝึกหัดไปแล้ว แต่โมเดลก็ยังจัดการกับความ Imbalance นี้ได้ไม่ดีพอจึงมีความแม่นยำต่ำกว่า Baseline อยู่

### ทำอะไรได้บ้างเพื่อให้คะแนนต่าง ๆ มีค่าสูงขึ้น และ/หรือ โมเดลมีประโยชน์มากขึ้น

### ปร​ะเด็น​อื่น ๆ ที่เกี่ยว​ข้องกั​บข้อมูลของ​ตัวเอง

### ก​าร​มีส่วน​ร่วมของสม​าชิกแต่ละ​คนในกลุ่ม (แต่ล​ะคนทำอะไรบ้าง)

### กา​รเปิ​ดเผ​ยกา​รใช้เค​รื่อ​งมือปัญ​ญาป​ระดิษฐ์ (ใช้อะไร ใช้เพื่ออะไร ใช้อย่างไร, pro​mpt อย่างไร)

ในการทำงานครั้งนี้ ไม่ได้ใช้ปัญญาประดิษฐ์ในการทำงานแต่อย่างใด